# **Preprocessing of Data**

Data Preprocessing is the process of preparing the data for analysis. This is the first step in any machine learning model.


Please refer [this article](https://analyticsindiamag.com/get-started-preparing-data-machine-learning/) to read about whole Machine Learning Cycle.

Here in this simple tutorial we will learn to implement Data preprocessing to perform the following operations on a raw dataset:

* Dealing with missing data
* Dealing with categorical data
* Splitting the dataset into training and testing sets
* Scaling the features


## **Importing the pandas**

In [ ]:
!python -m pip install pip --upgrade --user -q
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn openpyxl --user -q

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

In [168]:
import pandas as pd

## **Loading the dataset**

In [169]:
dataset = pd.read_excel("age_salary.xlsx")

The data set used here is as simple as shown below:

In [170]:
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


**Note:**
The ‘nan’ you see in some cells of the dataframe denotes the missing fields

Lets take only two columns firstly

In [171]:
df = dataset[['Age', 'Salary']]

In [172]:
df.head()

,Age,Salary
0,44.0,72000.0
1,27.0,48000.0
2,30.0,54000.0
3,38.0,61000.0
4,40.0,NaN


Now that we have loaded our dataset lets play with it.

## **Classifying the dependent and Independent Variables**

Having seen the data we can clearly identify the dependent and independent factors.Here we just have 2 factors, age and salary.Salary is the dependent factor that changes with the independent factor age.Now let’s classify them programmatically.

In [173]:
X = df.iloc[:,:-1].values #Takes all rows of all columns except the last column
Y = df.iloc[:,-1].values # Takes all rows of the last column

> * X : independent variable set
> * Y : dependent variable set


The dependent and independent values are stored in different arrays. In case of multiple independent variables use `X = dataset.iloc[:,a:b].values ` where a is the starting range and b is the ending range (column indices). You can also specify the column indices in a list to select specific columns.

## **Dealing with Missing Data**

We have already noticed the missing fields in the data denoted by “nan”. Machine learning models cannot accommodate missing fields in the data they are provided with.So the missing fields must be filled with values that will not affect the variance of the data or make it more noisy.

In [174]:
from sklearn.impute import SimpleImputer
import numpy as np
imp = SimpleImputer(missing_values=np.nan, strategy="mean")
X = imp.fit_transform(X)
Y = Y.reshape(-1,1)
Y = imp.fit_transform(Y)
Y = Y.reshape(-1)

The scikit-learn library’s SimpleImputer Class allows us to impute the missing fields in a dataset with valid data. In the above code, we have used the default strategy for filling missing values which is the mean. The imputer can not be applied on 1D arrays and since Y is a 1D array, it needs to be converted to a compatible shape.The reshape functions allows us to reshape any array.The `fit_transform()` method will fit the imputer object and then transforms the arrays.

In [175]:
X

array([[44.        ],
       [27.        ],
       [30.        ],
       [38.        ],
       [40.        ],
       [35.        ],
       [38.77777778],
       [48.        ],
       [50.        ],
       [37.        ]])

In [176]:
Y

array([72000.        , 48000.        , 54000.        , 61000.        ,
       63777.77777778, 58000.        , 52000.        , 79000.        ,
       83000.        , 67000.        ])

## **Dealing with Categorical Data**

When dealing with large and real-world datasets, categorical data is almost inevitable.Categorical variables represent types of data which may be divided into groups. Examples of categorical variables are race, sex, age group, educational level etc. These variables often has letters or words as its values. Since machine learning models are all about numbers and calculations , these categorical variables need to be coded in to numbers. Having coded the categorical variable into numbers may  just not be enough.

For example, consider the dataset below with 2 categorical features nation and purchased_item. Let us assume that the dataset is a record of how age, salary and country of a person determine if an item is purchased or not.Thus purchased_item is the dependent factor and age, salary and nation are the independent factors.

In [177]:
dataset["Age"] = X
dataset["Salary"] = Y

In [178]:
dataset

,Country,Age,Salary,Purchased
0,France,44.000000,72000.000000,No
1,Spain,27.000000,48000.000000,Yes
2,Germany,30.000000,54000.000000,No
3,Spain,38.000000,61000.000000,No
4,Germany,40.000000,63777.777778,Yes
5,France,35.000000,58000.000000,Yes
6,Spain,38.777778,52000.000000,No
7,France,48.000000,79000.000000,Yes
8,Germany,50.000000,83000.000000,No
9,France,37.000000,67000.000000,Yes


It has 3 countries listed. In a larger dataset, these may be large groups of data. Since countries don’t have a mathematical relation between them unless we are considering some known factors such as size or population etc , coding them in numbers will not work, as a number may be less than or greater than another number. Dummy variables are the solution. Using one hot encoding we will create a dummy variable for each of the category in the column. And uses binary encoding for each dummy variable. We do not need to create dummy variables for the feature purchased_item as it has only 2 categories either yes or no.

In [179]:
X = pd.DataFrame(dataset.iloc[:,[0,1,2]].values)
Y = dataset.iloc[:,3].values

from sklearn.preprocessing import LabelEncoder,OneHotEncoder
le_X = LabelEncoder()
X.iloc[:,0] = le_X.fit_transform(X.iloc[:,0])
country_one_hot_encode = pd.get_dummies(dataset.Country, prefix='Country')

In [180]:
X.drop([0], axis = 1, inplace=True)
X = pd.concat([X,country_one_hot_encode], axis=1)

In [181]:
X

,1,2,Country_France,Country_Germany,Country_Spain
0,44,72000,1,0,0
1,27,48000,0,0,1
2,30,54000,0,1,0
3,38,61000,0,0,1
4,40,63777.8,0,1,0
5,35,58000,1,0,0
6,38.7778,52000,0,0,1
7,48,79000,1,0,0
8,50,83000,0,1,0
9,37,67000,1,0,0


The the last 3 columns are the dummy features representing Germany,India and Russia respectively.The 1’s in each column represent that the person belongs to that specific country.

In [188]:
Y = le_X.fit_transform(Y)

In [189]:
Y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

## **Splitting the Dataset into Training and Testing sets**

All machine learning models require us to provide a training set for the machine so that the model can train from that data to understand the relations between features and can predict for new observations.When we are provided a single huge dataset with too much of observations ,it is a good idea to split the dataset into to two, a training_set and a test_set, so that we can test our model after its been trained with the training_set.

Scikit-learn comes with a method called train_test_split to help us with this task.

In [190]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 0)

The above code will split X and Y into two subsets each.

> * test_size: the desired size of the test_set. 0.3 denotes 30%.
> * random_state:  This is used to preserve the uniqueness. The split will happen uniquely for a random_state.


## **Scaling the features**

Since machine learning models rely on numbers to solve relations it is important to have similarly scaled data in a dataset. Scaling ensures that all data in a dataset falls in the same range.Unscaled data can cause inaccurate or false predictions.Some machine learning algorithms can handle feature scaling on its own and doesn’t require it explicitly.

The StandardScaler class from the scikit-learn library can help us scale the dataset.

In [191]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

sc_y = StandardScaler()
Y_train = Y_train.reshape((len(Y_train), 1))
Y_train = sc_y.fit_transform(Y_train)
Y_train = Y_train.ravel()

To read more about it, refer [here](https://analyticsindiamag.com/data-pre-processing-in-python/).